# MeLi scraping

In [ ]:
#! pip install requests
#! pip install beautifulsoup4
#! pip install matplotlib
#! pip install ipykernel

In [1]:
import utils_meli
from importlib import reload
import pandas as pd
import re
from datetime import datetime
import plotly.express as px

from IPython.display import display
pd.options.display.max_colwidth = 170
pd.options.display.max_rows = None

In [4]:
reload(utils_meli)

<module 'utils_meli' from 'c:\\Users\\nicarang\\Documents\\Digitas\\P&G\\Benchmarking\\utils_meli.py'>

In [2]:
# 30 min aprox.
df = utils_meli.load_html_search()

Response status code: 200
1
***************Shampoo Para La Caspa | MercadoLibre 📦***************
Total pages: 33
Current page: 1


Total products retrieved: 1657


,Brand,Product,Price,Link,Score,Ratings,Comments
0,ISDIN,Isdin Nutradeica Shampoo Anticaspa Grasa 200ml,434,https://www.mercadolibre.com.mx/isdin-nutradeica-shampoo-anticaspa-grasa-200ml/p/MLM19702444?pdp_filters=category:MLM430117#searchVariation=MLM19702444&position=1&sea...,4.7,73,[Pues en mi opinión nadie me lo recetó yo busque un shampoo porque tenía demasiada caspa y ya asta me sentía desesperada y mal de verme tanta cosas blancas en mi cabe...
1,MD LIFERPAL,Tiniazol Shampoo Alopecia Seborrea Y/o Caspa,48,https://www.mercadolibre.com.mx/tiniazol-shampoo-alopecia-seborrea-yo-caspa/p/MLM19505006?pdp_filters=category:MLM430117#searchVariation=MLM19505006&position=2&search...,4.6,50,"[Excelente para tratar dermatitis seborreica, desde el primer o segundo uso notas de inmediato el cambio. No duden, este si es tratamiento 100% efectivo!!., A la sema..."
2,ISDIN,Shampoo Isdin Lambdapil Anticaída en botella de 200mL,349,https://www.mercadolibre.com.mx/shampoo-isdin-lambdapil-anticaida-en-botella-de-200ml/p/MLM19147966?pdp_filters=category:MLM430117#searchVariation=MLM19147966&positio...,0.0,0,[[]]
3,-,Shampoo Selsun Medicated Max Strength Dandruff 325ml,299,https://articulo.mercadolibre.com.mx/MLM-1828433089-shampoo-selsun-medicated-max-strength-dandruff-325ml-_JM#position=33&search_layout=stack&type=item&tracking_id=f18...,5.0,5,"[Es muy bueno para el pelo grasoso y elimina la caspa y la comezón., Me ha sido muy útil y efectivo, desde el primer uso., Muy bueno gracias.]"
4,-,2 Shampoo Selsun Blue Medicated 325ml C/u,498,https://articulo.mercadolibre.com.mx/MLM-1403512770-2-shampoo-selsun-blue-medicated-325ml-cu-_JM#position=34&search_layout=stack&type=item&tracking_id=f1804ad5-e0e5-4...,5.0,8,"[Satisfecho con mi compra., Excelente articulo de calidad tal y como se publica., Excelente shampoo es un basico para mi.]"


In [23]:
df.to_excel("C:/Users/nicarang/Documents/Digitas/P&G/Benchmarking/H&S/Severe/shampoo_para_la_caspa_{}_{}_{}.xlsx".format(datetime.today().year,datetime.today().month,datetime.today().day) , index=False)

In [ ]:
# Pass the DataFrame and copy and paste the product name you want to see in browser
reload(utils_meli)
utils_meli.open_link(df,'Kit Capilar Argan Oil: Crema Rizadora Gd3 + Aceite Babyliss')

In [ ]:
# Pass the DataFrame and index you want to open in browser
reload(utils_meli)
utils_meli.open_link(df,3)

# Metrics

## Price Analysis

In [21]:
r"C:\Users\nicarang\Documents\Digitas\P&G\Benchmarking\H&S\Severe\MeLi\shampoo_para_la_caspa_2023_3_27.xlsx".replace("\\", "/")

'C:/Users/nicarang/Documents/Digitas/P&G/Benchmarking/H&S/Severe/MeLi/shampoo_para_la_caspa_2023_3_27.xlsx'

In [22]:
df = pd.read_excel(r"C:\Users\nicarang\Documents\Digitas\P&G\Benchmarking\H&S\Severe\MeLi\shampoo_para_la_caspa_2023_3_27.xlsx".replace("\\", "/"))
df.head()

,Brand,Product,Price,Link
0,MEDICASP,Shampoo Medicasp Anticaspa Uso Diario 2 Botellas De 130ml,195,https://www.mercadolibre.com.mx/shampoo-medicasp-anticaspa-uso-diario-2-botellas-de-130ml/p/MLM19094907?pdp_filters=category:MLM430117#searchVariation=MLM19094907&pos...
1,VICHY,Shampoo Vichy Anticaspa Dercos Ds Cabello Graso 200 Ml,557,https://articulo.mercadolibre.com.mx/MLM-769613335-shampoo-vichy-anticaspa-dercos-ds-cabello-graso-200-ml-_JM#position=29&search_layout=stack&type=item&tracking_id=bf...
2,SELSUN BLUE,Shampoo Selsun Blue Naturals Dandruff 325ml 11oz,223,https://www.mercadolibre.com.mx/shampoo-selsun-blue-naturals-dandruff-325ml-11oz/p/MLM19546760?pdp_filters=category:MLM430117#searchVariation=MLM19546760&position=2&s...
3,ISDIN,Isdin Nutradeica Shampoo Anticaspa Grasa 200ml,358,https://www.mercadolibre.com.mx/isdin-nutradeica-shampoo-anticaspa-grasa-200ml/p/MLM19702444?pdp_filters=category:MLM430117#searchVariation=MLM19702444&position=3&sea...
4,ISDIN,Shampoo Isdin Lambdapil Anticaída en botella de 200mL,330,https://www.mercadolibre.com.mx/shampoo-isdin-lambdapil-anticaida-en-botella-de-200ml/p/MLM19147966?pdp_filters=category:MLM430117#searchVariation=MLM19147966&positio...


In [23]:
reload(utils_meli)
df_25 = utils_meli.quartile_prices(df)[0]
df_25_50 = utils_meli.quartile_prices(df)[1]
df_50_75 = utils_meli.quartile_prices(df)[2]
df_75 = utils_meli.quartile_prices(df)[3]
quartiles = utils_meli.quartile_prices(df)[4]
print(f'Median: {df.Price.median()}')
quartiles

Median: 449.0


,Price
count,1673.00
mean,662.25
std,662.15
min,2.00
25%,247.00
50%,449.00
75%,831.00
max,4156.00


In [16]:
# Products above quartile 75
print('Prime products [above quartile 75] -> {} products ({}%) '.format(df_75.Product.to_list().__len__(), round((df_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Premium products [between quartiles 50 & 75] -> {} products ({}%) '.format(df_50_75.Product.to_list().__len__(), round((df_50_75.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Mid products [between quartiles 25 & 50] -> {} products ({}%) '.format(df_25_50.Product.to_list().__len__(), round((df_25_50.Product.to_list().__len__() / df.__len__()) * 100, 2)))
print('Low products [below quartile 25 ] -> {} products ({}%) '.format(df_25.Product.to_list().__len__(), round((df_25.Product.to_list().__len__() / df.__len__()) * 100, 2)))

Prime products [above quartile 75] -> 418 products (24.99%) 
Premium products [between quartiles 50 & 75] -> 417 products (24.93%) 
Mid products [between quartiles 25 & 50] -> 419 products (25.04%) 
Low products [below quartile 25 ] -> 419 products (25.04%) 


In [17]:
print('Quartile (75>) - Products: {}'.format(df_75.__len__()))
display(df_75.head())
display(px.bar(df_75.Brand.value_counts().to_frame()[df_75.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

print('Quartile (50-75] - Products: {}'.format(df_50_75.__len__()))
display(df_50_75.head())
display(px.bar(df_50_75.Brand.value_counts().to_frame()[df_50_75.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

print('Quartile (25-50] - Products: {}'.format(df_25_50.__len__()))
display(df_25_50.head())
display(px.bar(df_25_50.Brand.value_counts().to_frame()[df_25_50.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

print('Quartile (<25] - Products: {}'.format(df_25.__len__()))
display(df_25.head())
display(px.bar(df_25.Brand.value_counts().to_frame()[df_25.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results'))

Quartile (75>) - Products: 418


,Brand,Product,Price,Link
0,-,Shampoo Alquitran D' Hulla 20lt Tratamiento Psoriasis/ Hongo,1450,https://click1.mercadolibre.com.mx/mclics/clicks/external/MLM/count?a=jy3ff5s61ZKQtO7sVArbaoZ11r6KhLvZdtXidXHHAk5hsVY6VdSAFSM34HH5yJXDxRX8yQ7wvuzFfzM57bnOQ%2FpebgLd31...
1,JUSSE,Kit 3 Shampoo Jusse Alopecia Caída Crecimiento Frágil Frizz,999,https://click1.mercadolibre.com.mx/mclics/clicks/external/MLM/count?a=qTNjUzqT4FqbM9ZGSujQTX2QY%2BWZoM3FSdjJtsESYW3kP76jFcLc31wcH69gstVuDCmmKyZG6kWHfsjbNgmgEp5xAo0z19...
2,NEUTROGENA,Shampoo Gel Neutrogena T/gel Fórmula Original Pak 2 (250ml),875,https://articulo.mercadolibre.com.mx/MLM-1425050167-shampoo-gel-neutrogena-tgel-formula-original-pak-2-250ml-_JM#position=38&search_layout=stack&type=item&tracking_id...
3,PALMOLIVE,1152 Pzs/10ml. Shampoo Hotelero Palmolive Optims En Sobres,1709,https://articulo.mercadolibre.com.mx/MLM-871879796-1152-pzs10ml-shampoo-hotelero-palmolive-optims-en-sobres-_JM#position=41&search_layout=stack&type=item&tracking_id=...
4,-,Cln Champú Cuero Cabelludo Propensos Foliculitis Caspa Picor,1799,https://articulo.mercadolibre.com.mx/MLM-1518769756-cln-champu-cuero-cabelludo-propensos-foliculitis-caspa-picor-_JM#position=20&search_layout=stack&type=item&trackin...


Quartile (50-75] - Products: 417


,Brand,Product,Price,Link
0,VICHY,Shampoo Vichy Anticaspa Dercos Ds Cabello Graso 200 Ml,557,https://articulo.mercadolibre.com.mx/MLM-769613335-shampoo-vichy-anticaspa-dercos-ds-cabello-graso-200-ml-_JM#position=29&search_layout=stack&type=item&tracking_id=bf...
1,VICHY,Shampoo Vichy Anticaspa Dercos Ds Cabello Graso 200 Ml,557,https://click1.mercadolibre.com.mx/mclics/clicks/external/MLM/count?a=ox54sra9enQ5i0HHEMIKbYPspVU3q6zyVousvf9gcsISSujvrcWZAbKSYntDlFiB63Nes4rgUn2bcL8b0RQjPea1Vob3F2Gr...
2,DUCRAY,Shampoo Ducray Kelual DS en botella de 100mL por 1 unidad de 100mL,475,https://www.mercadolibre.com.mx/shampoo-ducray-kelual-ds-en-botella-de-100ml-por-1-unidad-de-100ml/p/MLM17897484?pdp_filters=category:MLM430117#searchVariation=MLM178...
3,MEDIHEALTH,"Betapirox Champú, Anti-caspa Medihealth 120 Ml, Pack 2",499,https://articulo.mercadolibre.com.mx/MLM-1366869309-betapirox-champu-anti-caspa-medihealth-120-ml-pack-2-_JM#position=36&search_layout=stack&type=item&tracking_id=bfe...
4,LA ROCHE-POSAY,Shampoo La Roche Posay Anticaspa Persistente Kerium 125ml,484,https://www.mercadolibre.com.mx/shampoo-la-roche-posay-anticaspa-persistente-kerium-125ml/p/MLM19569662?pdp_filters=category:MLM430117#searchVariation=MLM19569662&pos...


Quartile (25-50] - Products: 419


,Brand,Product,Price,Link
0,ISDIN,Isdin Nutradeica Shampoo Anticaspa Grasa 200ml,358,https://www.mercadolibre.com.mx/isdin-nutradeica-shampoo-anticaspa-grasa-200ml/p/MLM19702444?pdp_filters=category:MLM430117#searchVariation=MLM19702444&position=3&sea...
1,ISDIN,Shampoo Isdin Lambdapil Anticaída en botella de 200mL,330,https://www.mercadolibre.com.mx/shampoo-isdin-lambdapil-anticaida-en-botella-de-200ml/p/MLM19147966?pdp_filters=category:MLM430117#searchVariation=MLM19147966&positio...
2,SELSUN BLUE,Shampoo Selsun Medicated Max Strength Dandruff 325ml Importa,299,https://www.mercadolibre.com.mx/shampoo-selsun-medicated-max-strength-dandruff-325ml-importa/p/MLM19508792?pdp_filters=category:MLM430117#searchVariation=MLM19508792&...
3,-,Shampoo Selsun Medicated Max Strength Dandruff 325ml,299,https://articulo.mercadolibre.com.mx/MLM-1828433089-shampoo-selsun-medicated-max-strength-dandruff-325ml-_JM#position=33&search_layout=stack&type=item&tracking_id=bfe...
4,DUCRAY,Ducray Elution Shampoo Reequilibrante Cuero Cabelludo 200ml,298,https://www.mercadolibre.com.mx/ducray-elution-shampoo-reequilibrante-cuero-cabelludo-200ml/p/MLM19701625?pdp_filters=category:MLM430117#searchVariation=MLM19701625&p...


Quartile (<25] - Products: 419


,Brand,Product,Price,Link
0,MEDICASP,Shampoo Medicasp Anticaspa Uso Diario 2 Botellas De 130ml,195,https://www.mercadolibre.com.mx/shampoo-medicasp-anticaspa-uso-diario-2-botellas-de-130ml/p/MLM19094907?pdp_filters=category:MLM430117#searchVariation=MLM19094907&pos...
1,SELSUN BLUE,Shampoo Selsun Blue Naturals Dandruff 325ml 11oz,223,https://www.mercadolibre.com.mx/shampoo-selsun-blue-naturals-dandruff-325ml-11oz/p/MLM19546760?pdp_filters=category:MLM430117#searchVariation=MLM19546760&position=2&s...
2,-,Shampoo Alquitrán D Hulla Control Psoriasis Caspa 1 Litro,119,https://articulo.mercadolibre.com.mx/MLM-1350885680-shampoo-alquitran-d-hulla-control-psoriasis-caspa-1-litro-_JM#position=32&search_layout=stack&type=item&tracking_i...
3,CELESTY,Shampoo Alquitrán De Hulla Control Psoriasis Y Caspa 1 Litro,168,https://articulo.mercadolibre.com.mx/MLM-859722097-shampoo-alquitran-de-hulla-control-psoriasis-y-caspa-1-litro-_JM#position=34&search_layout=stack&type=item&tracking...
4,HEAD & SHOULDERS,Shampoo Head & Shoulders Men Con Fragancia Old Spice 1 L,159,https://www.mercadolibre.com.mx/shampoo-head-shoulders-men-con-fragancia-old-spice-1-l/p/MLM19505194?pdp_filters=category:MLM430117#searchVariation=MLM19505194&positi...


In [18]:
px.box(df,'Price', title=f'Price distribution')

In [323]:
print(df[df['Price'] <= 831].shape[0])
print(df[df['Price'] >= 449].shape[0] / df.shape[0])
print(df[(df['Price'] > 247) & (df['Price'] <= 449)].shape[0] / df.shape[0])

1255
0.5050806933652122
0.25044829647340106


In [94]:
display(px.histogram(df, x='Price', title=f'Price distribution'))
display(px.histogram(df_75, x='Price', title=f'Price distribution - Prime Products', nbins=10, text_auto=True))
display(df_75.describe().round(2))
display(px.histogram(df_50_75, x='Price', title=f'Price distribution - Premium Products', nbins=15, text_auto=True))
display(df_50_75.describe().round(2))
display(px.histogram(df_25_50, x='Price', title=f'Price distribution - Mid Products', nbins=15, text_auto=True))
display(df_25_50.describe().round(2))

,Price
count,418.00
mean,1565.33
std,741.13
min,833.00
25%,999.25
50%,1312.50
75%,1782.25
max,4156.00


,Price
count,417.00
mean,599.59
std,103.83
min,450.00
25%,507.00
50%,580.00
75%,680.00
max,831.00


,Price
count,419.00
mean,333.24
std,60.09
min,248.00
25%,289.50
50%,310.00
75%,395.00
max,449.00


In [316]:
#Prime brands
display(df_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10))
'''for i in df_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():
    print(i)'''

#Premium brands
display(df_50_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10))
'''for i in df_50_75.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():
    print(i)'''

#Mid brands
display(df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10))
'''for i in df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():
    print(i)'''

,Brand,Results
0,CELESTY,131
1,-,111
2,FRANCIS,98
3,PRODUCTOS MART MEXICO,7
4,NEUTROGENA,6
5,PALMOLIVE,6
6,NIZORAL,5
7,JUSSE,5
8,HEAD & SHOULDERS,5
9,TRUSS,4


,Brand,Results
0,-,122
1,FRANCIS,82
2,CELESTY,25
3,DUCRAY,16
4,LA ROCHE-POSAY,13
5,MEDIHEALTH,10
6,NEUTROGENA,8
7,NBC,8
8,VICHY,7
9,NIZORAL,7


,Brand,Results
0,-,161
1,FRANCIS,39
2,CELESTY,21
3,NBC,17
4,HEAD & SHOULDERS,12
5,SALERM,10
6,JOSS FERVIER,8
7,DUCRAY,8
8,SELSUN BLUE,8
9,SHELÓ NABEL,7


"for i in df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10).Brand.tolist():\n    print(i)"

In [315]:
a = df_25_50.Brand.value_counts().to_frame().reset_index().rename(columns={'index':'Brand','Brand':'Results'}).head(10)
a['Proc'] = round((a.Results / a.Results.sum())*100,2)
a

,Brand,Results,Proc
0,-,161,55.33
1,FRANCIS,39,13.40
2,CELESTY,21,7.22
3,NBC,17,5.84
4,HEAD & SHOULDERS,12,4.12
5,SALERM,10,3.44
6,JOSS FERVIER,8,2.75
7,DUCRAY,8,2.75
8,SELSUN BLUE,8,2.75
9,SHELÓ NABEL,7,2.41


## Brands distribution

In [247]:
df.columns

Index(['Brand', 'Product', 'Price', 'Link'], dtype='object')

In [248]:
px.bar(df.Brand.value_counts().to_frame()[df.Brand.value_counts().to_frame()['Brand']>=10], x='Brand', width=1100, height=600, text_auto=True,
       labels={'index':'Brand', 'Brand':'Count'}).update_layout(xaxis={'categoryorder':'total descending'}, title='Brands over 10 search results')

In [227]:
print(df[df.Brand!='-'].Brand.unique().tolist())

['MEDICASP', 'VICHY', 'SELSUN BLUE', 'ISDIN', 'CELESTY', 'HEAD & SHOULDERS', 'DUCRAY', 'IMAGES', 'MEDIHEALTH', 'JUSSE', 'MU&ME', 'LA ROCHE-POSAY', 'NEUTROGENA', 'PRODUCTOS MART MEXICO', 'HERMOSO REMEDIO', 'DISNEY', 'ALERT', 'PALMOLIVE', 'CANTABRIA', 'BIODERMA', 'SILANES', 'DS LABORATORIES', 'SELSUN', 'NATURA', 'FOLICURE', 'EGO', 'URIAGE', 'YAKARTA', 'NBC', 'SALERM', 'YVES ROCHER', 'IRALTONE', 'MD LIFERPAL', 'EUDERMA', 'ALEA', 'ACM', "FEMME D'OR", 'NEFERTITI', 'SISTEMA GB', 'MANDUMED GRANJA ECOLÓGICA', 'SEDAL', 'FOLCRESS', 'REDKEN', 'YEGUADA LA RESERVA', 'NOPSOR', 'GRISI', 'KÉRASTASE', 'LABORATORIOS FRANCIS', 'DAVINES', 'TRUSS', 'BOTANICUS', 'SHELÓ NABEL', 'VAERÁ', 'LAZARTIGUE', 'PRIMED', 'NEO CAPELLI PRO', 'INCREDIBLE', 'SECRETOS DE ANTAÑO', 'LACER', 'DEL INDIO PAPAGO', 'ICON', 'SKIN-CAP', 'NIZORAL', 'CAPRICE', 'SALERM COSMETICS', 'VOGLIA', 'SCHWARZKOPF', 'PG', 'ELIX', 'LA ROCHE POSAY', 'FARMAPIEL', 'GENOVÉ', 'FRANCIS', 'MATRIX', 'SKIN DER', 'KANIN', 'MAGNUS', 'MULTIPLUS', 'DR. KURT WO

In [249]:
test = df.head(10).copy()
test

,Brand,Product,Price,Link
0,MEDICASP,Shampoo Medicasp Anticaspa Uso Diario 2 Botellas De 130ml,195,https://www.mercadolibre.com.mx/shampoo-medicasp-anticaspa-uso-diario-2-botellas-de-130ml/p/MLM19094907?pdp_filters=category:MLM430117#searchVariation=MLM19094907&pos...
1,VICHY,Shampoo Vichy Anticaspa Dercos Ds Cabello Graso 200 Ml,557,https://articulo.mercadolibre.com.mx/MLM-769613335-shampoo-vichy-anticaspa-dercos-ds-cabello-graso-200-ml-_JM#position=29&search_layout=stack&type=item&tracking_id=bf...
2,SELSUN BLUE,Shampoo Selsun Blue Naturals Dandruff 325ml 11oz,223,https://www.mercadolibre.com.mx/shampoo-selsun-blue-naturals-dandruff-325ml-11oz/p/MLM19546760?pdp_filters=category:MLM430117#searchVariation=MLM19546760&position=2&s...
3,ISDIN,Isdin Nutradeica Shampoo Anticaspa Grasa 200ml,358,https://www.mercadolibre.com.mx/isdin-nutradeica-shampoo-anticaspa-grasa-200ml/p/MLM19702444?pdp_filters=category:MLM430117#searchVariation=MLM19702444&position=3&sea...
4,ISDIN,Shampoo Isdin Lambdapil Anticaída en botella de 200mL,330,https://www.mercadolibre.com.mx/shampoo-isdin-lambdapil-anticaida-en-botella-de-200ml/p/MLM19147966?pdp_filters=category:MLM430117#searchVariation=MLM19147966&positio...
5,-,Shampoo Alquitrán D Hulla Control Psoriasis Caspa 1 Litro,119,https://articulo.mercadolibre.com.mx/MLM-1350885680-shampoo-alquitran-d-hulla-control-psoriasis-caspa-1-litro-_JM#position=32&search_layout=stack&type=item&tracking_i...
6,SELSUN BLUE,Shampoo Selsun Medicated Max Strength Dandruff 325ml Importa,299,https://www.mercadolibre.com.mx/shampoo-selsun-medicated-max-strength-dandruff-325ml-importa/p/MLM19508792?pdp_filters=category:MLM430117#searchVariation=MLM19508792&...
7,VICHY,Shampoo Vichy Anticaspa Dercos Ds Cabello Graso 200 Ml,557,https://click1.mercadolibre.com.mx/mclics/clicks/external/MLM/count?a=ox54sra9enQ5i0HHEMIKbYPspVU3q6zyVousvf9gcsISSujvrcWZAbKSYntDlFiB63Nes4rgUn2bcL8b0RQjPea1Vob3F2Gr...
8,-,Shampoo Alquitran D' Hulla 20lt Tratamiento Psoriasis/ Hongo,1450,https://click1.mercadolibre.com.mx/mclics/clicks/external/MLM/count?a=jy3ff5s61ZKQtO7sVArbaoZ11r6KhLvZdtXidXHHAk5hsVY6VdSAFSM34HH5yJXDxRX8yQ7wvuzFfzM57bnOQ%2FpebgLd31...
9,-,Shampoo Selsun Medicated Max Strength Dandruff 325ml,299,https://articulo.mercadolibre.com.mx/MLM-1828433089-shampoo-selsun-medicated-max-strength-dandruff-325ml-_JM#position=33&search_layout=stack&type=item&tracking_id=bfe...


In [543]:
'Shampoo Vichy Anticaspa Dercos Ds Cabello Graso 200 Ml'.lower().__contains__('vichy'.lower())

True

In [262]:
idx = 0
b = test[test.Brand!='-'].Brand.unique().tolist()
for i in test.Product:
    for j in b:
        if i.__contains__(j.lower()) | i.__contains__(j.upper()) | i.__contains__(j.title()):

            print(j)
        else: print('No')
        

MEDICASP
No
No
No
No
VICHY
No
No
No
No
SELSUN BLUE
No
No
No
No
ISDIN
No
No
No
ISDIN
No
No
No
No
No
No
No
No
No
VICHY
No
No
No
No
No
No
No
No
No
No


## Brand appearences

In [246]:
# Look for specif word(s)
count = 0
brand_producs = []
for i in df['Product']:
    if (re.search('head.', i, flags=re.IGNORECASE)) or (re.search('h&s.', i, flags=re.IGNORECASE)) or (re.search('head & shoulders.', i, flags=re.IGNORECASE)):
        count += 1
        brand_producs.append(i)
print(count)

brand_df = pd.DataFrame(data={'Product':brand_producs})
brand_df.head(10)

81


,Product
0,Shampoo Head & Shoulders Men Con Fragancia Old Spice 1 L
1,Shampoo Head & Shoulders 2 En 1 Suave Y Manejable 1 Lt
2,Shampoo Head & Shoulders 2 En 1 Suave Y Manejable 1 L
3,Shampoo Head & Shoulders Clinical Solutions
4,Shampoo Head & Shoulders 2 En 1 Limp Renovadora 1lt (2pack)
5,Shampoo Head & Shoulders Clinical Solutions Formula Renovada
6,Shampoo Head & Shoulders Men Con Fragancia Old Spice 1 Litro
7,576 Sobres/10ml. Shampoo Head & Shoulder Hotelero
8,Shampoo Head & Shoulder Hotelero 288 Sobres De 10ml
9,Shampoo 2 en 1 Head & Shoulders Suave Y Manejable en botella de 375mL por 1 unidad


In [267]:
count = 0
brand_producs = []
brand_prices = []
for i in range(df.__len__()):
    if (re.search('head.', df.loc[i,'Product'], flags=re.IGNORECASE)) or (re.search('h&s.', df.loc[i,'Product'], flags=re.IGNORECASE)) or (re.search('head & shoulders.', df.loc[i,'Product'], flags=re.IGNORECASE)):
        count += 1
        brand_producs.append(df.loc[i,'Product'])
        brand_prices.append(df.loc[i,'Price'])
print(count)

brand_df = pd.DataFrame(data={'Product':brand_producs, 'Price':brand_prices})
print(round(brand_df.Price.mean(), 2))
brand_df.head(10)

81
348.23


,Product,Price
0,Shampoo Head & Shoulders Men Con Fragancia Old Spice 1 L,159
1,Shampoo Head & Shoulders 2 En 1 Suave Y Manejable 1 Lt,148
2,Shampoo Head & Shoulders 2 En 1 Suave Y Manejable 1 L,149
3,Shampoo Head & Shoulders Clinical Solutions,495
4,Shampoo Head & Shoulders 2 En 1 Limp Renovadora 1lt (2pack),298
5,Shampoo Head & Shoulders Clinical Solutions Formula Renovada,498
6,Shampoo Head & Shoulders Men Con Fragancia Old Spice 1 Litro,159
7,576 Sobres/10ml. Shampoo Head & Shoulder Hotelero,1499
8,Shampoo Head & Shoulder Hotelero 288 Sobres De 10ml,875
9,Shampoo 2 en 1 Head & Shoulders Suave Y Manejable en botella de 375mL por 1 unidad,50


----------------------------------------

In [324]:
import pandas as pd
import webbrowser
import requests
import bs4
from IPython.display import display

In [23]:
search = input('Insert your search:')
url = 'https://listado.mercadolibre.com.mx/{}#D[A:{}]'.format(search.replace(' ','-'), search)
request = requests.get(url)
print(f'Response status code: {request.status_code}')
content = request.text
soup = bs4.BeautifulSoup(content, 'html.parser') # HTML document

total_pages = int(soup.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})[0].find(class_="andes-pagination__page-count").text.replace("de ",""))
current_page = soup.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})[0].find("li").text # See actual page
print(f'***************{soup.title.text}***************')
print('Total pages: {}'.format(total_pages))
print('Current page: {}\n\n'.format(current_page))

brands = []
products = []
prices = []
scores = []
comments = []
links = []

for page in range(1,total_pages+1):

    if page == 1:
        url_page = 'https://listado.mercadolibre.com.mx/{}_NoIndex_True'.format(search.replace(' ','-'))
        request_page = requests.get(url_page)
        content_page = request_page.text
        soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
        tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

        for tag in tag_filter_page:
            brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
            products.append(tag.find("h2").text)
            prices.append(tag.find(class_="price-tag-fraction").text)
            links.append(tag.find("a")["href"])

    else:
        url_page = 'https://listado.mercadolibre.com.mx/{}_Desde_{}_NoIndex_True'.format(search.replace(' ','-'),(page*50)+1)
        request_page = requests.get(url_page)
        content_page = request_page.text
        soup_page = bs4.BeautifulSoup(content_page, 'html.parser')
        tag_filter_page = soup_page.find_all("div", {"class":"ui-search-result__content-wrapper shops__result-content-wrapper"})

        for tag in tag_filter_page:
            brands.append(tag.find(class_="ui-search-item__brand-discoverability ui-search-item__group__element shops__items-group-details").text)
            products.append(tag.find("h2").text)
            prices.append(tag.find(class_="price-tag-fraction").text)
            links.append(tag.find("a")["href"])
        

data = {}
data["Brand"] = brands
data['Product'] = products
data['Price'] = prices
'''data['Score'] = scores
data['Comentarios'] = comments'''
data["Link"] = links

df = pd.DataFrame(data = data)
df = df.drop_duplicates(ignore_index=True)
df['Brand'] = df['Brand'].apply(lambda x: "-" if x == '' else x)
df['Price'] = df['Price'].apply(lambda x: x.replace(",",""))
df['Price'] = df['Price'].astype(int)
print('Total products retrieved: {}'.format(df.shape[0]))
display(df.head())

Response status code: 200
***************Crema Para Peinar Rizos | MercadoLibre 📦***************
Total pages: 19
Current page: 1


Total products retrieved: 998


,Brand,Product,Price,Link
0,CANTU,Crema Para Peinar Cantu Shea Butter Para Rizos 355ml,156,https://www.mercadolibre.com.mx/crema-para-peinar-cantu-shea-butter-para-rizos-355ml/p/MLM18977626?pdp_filters=category:MLM171893#searchVariation=MLM18977626&position...
1,KOLOR SHOT,Tratamiento Profesional Rizos Definidos Kolor Shot® 250 Ml,77,https://www.mercadolibre.com.mx/tratamiento-profesional-rizos-definidos-kolor-shot-250-ml/p/MLM19876322?pdp_filters=category:MLM171893#searchVariation=MLM19876322&pos...
2,CANTU,Crema Hidratante Activador De Rizos Cantu 12 Fl Oz/ 355 Ml,183,https://articulo.mercadolibre.com.mx/MLM-807592440-crema-hidratante-activador-de-rizos-cantu-12-fl-oz-355-ml-_JM#position=25&search_layout=stack&type=item&tracking_id...
3,CANTU,Crema Para Peinar Cantu Shea Butter Para Rizos 340g,144,https://www.mercadolibre.com.mx/crema-para-peinar-cantu-shea-butter-para-rizos-340g/p/MLM18977688?pdp_filters=category:MLM171893#searchVariation=MLM18977688&position=...
4,CANTU,Cantu Crema De Peinar Shea Butter Para Rizos 355ml,156,https://articulo.mercadolibre.com.mx/MLM-1498288314-cantu-crema-de-peinar-shea-butter-para-rizos-355ml-_JM#position=26&search_layout=stack&type=item&tracking_id=67323...


In [520]:
request_ = requests.get('https://listado.mercadolibre.com.mx/iphone-3#D[A:iphone%203]')
print(f'Response status code: {request_.status_code}')
content_ = request_.text
soup_ = bs4.BeautifulSoup(content_, 'html.parser') # HTML document
total_pages = soup_.find_all("div", {"class":"ui-search-pagination shops__pagination-content"})#[0].find(class_="andes-pagination__page-count").text.replace("de ",""))
print('Total pages: {}'.format(total_pages))
print(soup_.title.text)

Response status code: 200
Total pages: []
Iphone 3 | MercadoLibre 📦


In [527]:
soup_.find_all("div", {"class":"ui-search-pagination shops__pagination-content"}).__len__()

0

In [441]:
soup_.find_all('article', {"class":"ui-review-capability-comments__comment"})

[<article aria-roledescription="Review" class="ui-review-capability-comments__comment" data-testid="comment-component"><div class="ui-review-capability-comments__comment__header"><div class="ui-review-capability-comments__comment__rating-container"><div class="ui-review-capability-comments__comment__rating"><svg aria-hidden="true" class="ui-review-capability-comments__comment__rating__star" height="16" viewbox="0 0 10 10" width="16.8"><path d="M5.056 8L1.931 9.648l.597-3.49L0 3.684l3.494-.509L5.056 0l1.562 3.176 3.494.51-2.528 2.471.597 3.491z" fill="#3483FA" fill-rule="evenodd"></path></svg><svg aria-hidden="true" class="ui-review-capability-comments__comment__rating__star" height="16" viewbox="0 0 10 10" width="16.8"><path d="M5.056 8L1.931 9.648l.597-3.49L0 3.684l3.494-.509L5.056 0l1.562 3.176 3.494.51-2.528 2.471.597 3.491z" fill="#3483FA" fill-rule="evenodd"></path></svg><svg aria-hidden="true" class="ui-review-capability-comments__comment__rating__star" height="16" viewbox="0 0 1

In [494]:
v = []
l = soup_.find_all('article', {"class":"ui-review-capability-comments__comment"}) #[2].find('p', class_="ui-review-capability-comments__comment__content").text
print(l.__len__())
for c in l:
    v.append(c.find('p', class_="ui-review-capability-comments__comment__content").text)
print(l.__len__())
v

3
3


['Producto relativamente caro , pero hace muy bien su trabajo , se ve la diferencia en el cabello desde el primer uso, y con el uso continuo sin duda elimina el problema de la caspa , muy recomendado.',
 'El precio es muy bueno, excelente, el tratamiento me ha ayudado y es el único que me ayudado de verdad, tengo cabello rizado y la resequedad es un martirio, aunado a la caspa. Muy bien producto.',
 'Excelente producto cumple con lo prometido estoy tan sorprendida.']